# Probability and Sampling

In [ ]:
from datascience import *
from cs104 import *
import numpy as np
%matplotlib inline

## 1. Distributions ##

### Probability Distribution

We can use probability rules to analytically write down the expected number of each possible value in order to create a probability distribution like the following

In [ ]:
# Sums of all possible combinations of two dice rolls. 
# (The first few entries illustrate how we constructed these combinations.)
outcomes = make_array(1+1,1+2,2+1,1+3,2+2,3+1,5,5,5,5,6,6,6,6,6,
                      7,7,7,7,7,7,8,8,8,8,8,9,9,9,9,10,10,10,11,11,12)
outcome_bins = np.arange(1.5, 13.5, 1)
plot = Table().with_columns('Sum of two dice rolls', outcomes).hist(bins=outcome_bins)
plot.set_title('Probability (exact) distribution \n   ')
plot.set_ylim(0,0.175)

### Empirical Distribution

In [ ]:
dice = np.arange(1,7)
dice

Let's roll the dice twice and add the values. 

In [ ]:
two_dice = np.random.choice(dice, 2)
print('two dice=', two_dice)
print('sum=', sum(two_dice))

Let's put this together in a function that `simulate` can use as an input. 

In [ ]:
def sum_two_dice(): 
    dice = np.arange(1,7)
    two_dice = np.random.choice(dice, 2)
    return sum(two_dice)

Use `simulate` (from our inference library) to create an empirical distribution. 

In [ ]:
def simulate(make_one_outcome, num_trials):
    """
    Return an array of num_trials values, each 
    of which was created by calling make_one_outcome().
    """
    outcomes = make_array()
    for i in np.arange(0, num_trials):
        outcome = make_one_outcome()
        outcomes = np.append(outcomes, outcome)

    return outcomes

In [ ]:
num_trials = 10 
simulate(sum_two_dice, num_trials)

In [ ]:
num_trials = 2000 
all_outcomes = simulate(sum_two_dice, num_trials)

In [ ]:
simulated_results = Table().with_column('Sum of two dice rolls', all_outcomes)
simulated_results

In [ ]:
plot = simulated_results.hist(bins=outcome_bins)
plot.set_title('Empirical (approximate) distribution \n num_trials='+str(num_trials));
plot.set_ylim(0,0.175)

### Law of Averages

In our simulation, we have one parameter that we have the ability to control `num_trials`. Does this parameter matter? 

To find out, we can write a function that takes as input the `num_trials` parameter.

In [ ]:
def simulate_and_plot_summing_two_dice(num_trials):
    """
    Simulates rollowing two dice and repeats num_trials times, and 
    Plots the empirical distribution
    """
    all_outcomes = simulate(sum_two_dice, num_trials)
    simulated_results = Table().with_column('Sum of two dice rolls', all_outcomes)

    outcome_bins = np.arange(1.5, 13.5, 1)
    plot = simulated_results.hist(bins=outcome_bins)
    plot.set_title('Empirical (approximate) distribution \n num_trials='+str(num_trials))
    plot.set_ylim(0,0.18)

In [ ]:
simulate_and_plot_summing_two_dice(2000)

Here are a couple plots for different numbers of trials:

In [ ]:
with Figure(1,3, sharey=True):
    simulate_and_plot_summing_two_dice(100)
    simulate_and_plot_summing_two_dice(500)
    simulate_and_plot_summing_two_dice(20000)

In [ ]:
interact(simulate_and_plot_summing_two_dice, num_trials = Slider(1,1000))

## 2. Random Sampling: Florida Votes in 2016

Load data for voting in Florida in 2016.  These give us the true parameters if we were able to poll every person who would turn out to vote:

- Proportion voting for (Trump, Clinton, Johnson, other) = (0.49, 0.478, 0.022, 0.01)
- Raw counts:
    - Trump: 4,617,886
    - Clinton: 4,504,975
    - Johnson: 207,043
    - Other: 90,135

Data is based on the actual votes case in the election.

In [ ]:
votes = Table().read_table('data/florida_2016.csv')
votes = votes.with_column('Vote', votes.apply(make_array("Trump", "Clinton", "Johnson", "Other").item, "Vote"))

In [ ]:
votes.show(5)

Here's the total number of votes cast in the election. 

In [ ]:
votes.num_rows

We can pick a "convenience sample": the first 10 voters who show up in line.

In [ ]:
votes.take(np.arange(10))

Since we are analyzing this **after the election**, we actually know the votes for the full population and we can compute the true parameter.

In [ ]:
sum(votes.column('Vote') == 'Trump') / votes.num_rows

But suppose this is **before the election** and we actually can't ask every person in the state how they will vote...

In that case, we can imagine we are a pollster, and sample 50 people. 

We can use `.sample(n)` to randomly sample `n` rows from a table. 

In [ ]:
sample = votes.sample(50)
sample

In [ ]:
sum(sample.column('Vote') == 'Trump') / sample.num_rows

Let's write functions to do this! 
1. A function that takes a sample
2. A function that computes the statistic (proportion of the sample that voted for Trump). 

In [ ]:
def sample_votes(sample_size): 
    return votes.sample(sample_size)

In [ ]:
def proportion_vote_trump(sample): 
    return sum(sample.column('Vote') == 'Trump') / sample.num_rows

In [ ]:
sample = sample_votes(100)
proportion_vote_trump(sample)

In [ ]:
proportion_vote_trump(sample_votes(100))

In [ ]:
proportion_vote_trump(sample_votes(1000))

So far, we've been using a `simulate` function. Let's extend this to a function that can also take a sample size. We'll call this function `simulate_sample_statistic`. 

In [ ]:
def simulate_sample_statistic(make_one_sample, sample_size,
                              compute_sample_statistic, num_trials):
    """
    Simulates num_trials sampling steps and returns an array of the
    statistic for those samples.  The parameters are:

    - make_one_sample: a function that takes an integer n and returns a 
                   sample as an array of n elements.
    
    - sample_size: the size of the samples to use in the simulation.
    
    - compute_statistic: a function that takes a sample as 
                         an array and returns the statistic for that sample. 
    
    - num_trials: the number of simulation steps to perform.
    """

    simulated_statistics = make_array()
    for i in np.arange(0, num_trials):
        simulated_sample = make_one_sample(sample_size)
        sample_statistic = compute_sample_statistic(simulated_sample)
        simulated_statistics = np.append(simulated_statistics, sample_statistic)
    return simulated_statistics

Let's use our simulation algorithm to create an empirical distribution. 

Suppose there are 1,000 polling companies and each uses a sample of 100 people. 

In [ ]:
num_trials = 1000 #1,000 polling companies
sample_size = 100 #100 people sampled by each polling company 

all_outcomes = simulate_sample_statistic(sample_votes, sample_size,
                                         proportion_vote_trump, num_trials)
all_outcomes

In [ ]:
simulated_results = Table().with_column('Proportion voting for Trump', all_outcomes)
plot = simulated_results.hist()

title = 'Empirical (approximate) distribution \n num_trials='+str(num_trials)+ '\n sample_size='+str(sample_size)
plot.set_title(title)

Let's make a function with our two free parameters, `num_trials` and `sample_size`. 

In [ ]:
def simulate_and_plot_trump_pollster(num_trials, sample_size): 
    all_outcomes = simulate_sample_statistic(sample_votes, sample_size,
                        proportion_vote_trump, num_trials)
    simulated_results = Table().with_column('Proportion voting for Trump', all_outcomes)
    plot = simulated_results.hist(bins=np.arange(0.3,0.71,0.025))
    title = 'Empirical (approximate) distribution \n num_trials='+str(num_trials)+ '\n sample_size='+str(sample_size)
    plot.set_title(title)    

Here are a few choices for parameters.  Notice how each impacts the resulting histogram.

In [ ]:

with Figure(2,2, sharey=True, sharex=True):
    import matplotlib.pyplot as plots
    simulate_and_plot_trump_pollster(100, 200)
    simulate_and_plot_trump_pollster(100, 1000)
    simulate_and_plot_trump_pollster(5000, 200)
    simulate_and_plot_trump_pollster(5000, 1000)
    plots.xlim(0.3, 0.7)
    plots.ylim(0,30)    

In [ ]:
interact(simulate_and_plot_trump_pollster, 
         num_trials = Choice(make_array(1,10,100,1000,5000)), 
         sample_size = Choice(make_array(1,10,100,1000,5000)))

**Big picture questions sampling**: 
- Why wouldn't we always just take really big of samples since they converge to the true distribution?

**Big picture questions simulations**: 
- What are we abstracting away when we're writing code? What are we re-using over and over? 